In [2]:
import pandas as pd

In [257]:
cities = pd.read_json("data/italy_cities.json")
geo = pd.read_json("data/italy_geo.json")
provinces = pd.read_json("data/italy_provincies.json")

In [258]:
cities_geo = cities.join(geo, lsuffix="_city").drop(["istat_city", "comune_city", "prefisso", "cod_fisco", "superficie"], axis=1)

In [259]:
cities_geo

,regione,provincia,num_residenti,istat,comune,lng,lat
0,Piemonte,TO,2669.0,1001,Agliè,7.7686,45.363433
1,Piemonte,TO,3842.0,1002,Airasca,7.48443104,44.916886
2,Piemonte,TO,469.0,1003,Ala di Stura,7.30434392,45.315110
3,Piemonte,TO,1800.0,1004,Albiano d'Ivrea,7.94914491,45.433893
4,Piemonte,TO,712.0,1005,Alice Superiore,7.77701858,45.460094
...,...,...,...,...,...,...,...
7975,Sardegna,SU,3439.0,111105,Villasimius,9.52036544,39.142384
7976,Sardegna,SU,6875.0,111106,Villasor,8.94065031,39.380689
7977,Sardegna,SU,2437.0,111107,Villaspeciosa,8.92693384,39.312805
7978,,,59620754.0,Total,,,7978.000000


In [260]:
cities_geo.loc[3802,"regione"]  = "Emilia-Romagna"

cities_geo.drop([7978,7979], axis=0, inplace=True)

In [261]:
cities_geo

,regione,provincia,num_residenti,istat,comune,lng,lat
0,Piemonte,TO,2669.0,1001,Agliè,7.7686,45.363433
1,Piemonte,TO,3842.0,1002,Airasca,7.48443104,44.916886
2,Piemonte,TO,469.0,1003,Ala di Stura,7.30434392,45.315110
3,Piemonte,TO,1800.0,1004,Albiano d'Ivrea,7.94914491,45.433893
4,Piemonte,TO,712.0,1005,Alice Superiore,7.77701858,45.460094
...,...,...,...,...,...,...,...
7973,Sardegna,SU,4811.0,111103,Villaputzu,9.57483207,39.440086
7974,Sardegna,SU,1107.0,111104,Villasalto,9.38877379,39.493758
7975,Sardegna,SU,3439.0,111105,Villasimius,9.52036544,39.142384
7976,Sardegna,SU,6875.0,111106,Villasor,8.94065031,39.380689


In [262]:
x = cities_geo[['lat', 'lng']].values.tolist()

coords = []
for lat, lng in x:
    coords.append(f"({lat}, {float(lng)})")

cities_geo["coords"] = coords
cities_geo.drop(["lng","lat"], axis=1, inplace=True)
cities_geo

,regione,provincia,num_residenti,istat,comune,coords
0,Piemonte,TO,2669.0,1001,Agliè,"(45.36343304, 7.7686)"
1,Piemonte,TO,3842.0,1002,Airasca,"(44.91688598, 7.48443104)"
2,Piemonte,TO,469.0,1003,Ala di Stura,"(45.31511014, 7.30434392)"
3,Piemonte,TO,1800.0,1004,Albiano d'Ivrea,"(45.43389336, 7.94914491)"
4,Piemonte,TO,712.0,1005,Alice Superiore,"(45.4600936, 7.77701858)"
...,...,...,...,...,...,...
7973,Sardegna,SU,4811.0,111103,Villaputzu,"(39.44008566, 9.57483207)"
7974,Sardegna,SU,1107.0,111104,Villasalto,"(39.49375831, 9.38877379)"
7975,Sardegna,SU,3439.0,111105,Villasimius,"(39.14238405, 9.52036544)"
7976,Sardegna,SU,6875.0,111106,Villasor,"(39.38068871, 8.94065031)"


In [263]:
provinces.rename(columns = {"sigla":"provincia","provincia":"nome"}, inplace=True)

In [264]:
columns_to_del = ["superficie", "residenti", "num_comuni", "id_regione"]
provinces.drop(columns_to_del, axis=1, inplace=True)
provinces.drop([107,108], axis=0, inplace=True)
provinces

,provincia,nome
0,AG,Agrigento
1,AL,Alessandria
2,AN,Ancona
3,AO,Aosta
4,AP,Ascoli Piceno
...,...,...
102,VE,Venezia
103,VI,Vicenza
104,VR,Verona
105,VT,Viterbo


In [166]:
import psycopg2

In [265]:
provinces = provinces.to_numpy().tolist()

In [182]:
try:
    conn = psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PSWD)
    
    cur = conn.cursor()
    
    sql = "INSERT INTO provinces(province, name) VALUES(%s, %s)"
    cur.executemany(sql,provinces)
    # commit the changes to the database
    conn.commit()
    # close communication with the database
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

In [269]:
try:
    conn = conn = psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PSWD)
    
    cur = conn.cursor()
    sql = "INSERT INTO municipalities(region, province, population, istat, municipality, coords) VALUES"#(%s, %s, %s, %s, %s, %s::point)"
    args_str = ','.join(cur.mogrify("(%s,%s,%s,%s,%s,%s::point)", x).decode("utf-8") for x in cities_geo.values.tolist())
    # for city in cities_geo.values.tolist():
    cur.execute(sql+args_str)# city)
    # commit the changes to the database
    conn.commit()
    # close communication with the database
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()